In [0]:
import tensorflow as tf
import numpy as np

In [0]:
sample = " Mulmi team, cheer-up"
# "Mulmi team, cheer-u" => "Mulmi team, cheer-up"

In [28]:
len(sample)

21

In [11]:
set(sample) # 중복 제거

{' ', ',', '-', 'M', 'a', 'c', 'e', 'h', 'i', 'l', 'm', 'p', 'r', 't', 'u'}

In [0]:
idx2char = list(set(sample))
char2idx = { c:i for i,c in enumerate(idx2char)}

In [13]:
char2idx

{' ': 4,
 ',': 6,
 '-': 5,
 'M': 13,
 'a': 3,
 'c': 11,
 'e': 7,
 'h': 1,
 'i': 9,
 'l': 8,
 'm': 14,
 'p': 12,
 'r': 10,
 't': 2,
 'u': 0}

In [0]:
dic_size = len(char2idx)
hidden_size = num_classes = dic_size
# dic_size : OHE size
# hidden_size = output size
# num_classes = final output size

In [19]:
dic_size

15

In [9]:
print(num_classes)

15


In [0]:
batch_size = 1 # data 1개 => batch => update
sequence_length = len(sample) - 1
learning_rate = 0.1

In [0]:
sample_idx = [char2idx[c] for c in sample]

In [168]:
sample_idx

[4, 13, 0, 8, 14, 9, 4, 2, 7, 3, 14, 6, 4, 11, 1, 7, 7, 10, 5, 0, 12]

In [169]:
sample_idx[:-1] # 맨 마지막 거 지운거! => input

[4, 13, 0, 8, 14, 9, 4, 2, 7, 3, 14, 6, 4, 11, 1, 7, 7, 10, 5, 0]

In [0]:
tf.reset_default_graph()

xdata = [sample_idx[:-1]]
ydata = [sample_idx[1:]]
x = tf.placeholder(tf.int32, [None, sequence_length])
y = tf.placeholder(tf.int32, [None, sequence_length])
x_one_hot = tf.one_hot(x, num_classes)
cell = tf.contrib.rnn.BasicLSTMCell(
  num_units = hidden_size)
ini_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=ini_state)

In [213]:
outputs

<tf.Tensor 'rnn/transpose_1:0' shape=(1, 20, 15) dtype=float32>

In [0]:
xfc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(xfc, num_classes, activation_fn=None)

In [215]:
x

<tf.Tensor 'Placeholder:0' shape=(?, 20) dtype=int32>

In [216]:
x_one_hot

<tf.Tensor 'one_hot:0' shape=(?, 20, 15) dtype=float32>

In [217]:
outputs

<tf.Tensor 'fully_connected/BiasAdd:0' shape=(20, 15) dtype=float32>

In [0]:
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [219]:
outputs

<tf.Tensor 'Reshape_1:0' shape=(1, 20, 15) dtype=float32>

In [0]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y, weights=weights)

loss = tf.reduce_mean(sequence_loss)

train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, 2)

In [221]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(200):
    cv, _ = sess.run([loss, train], feed_dict={x:xdata, y:ydata})
    print(i, cv)
    res = sess.run(prediction, feed_dict={x:xdata})
    res_str = [idx2char[c] for c in np.squeeze(res)]
    print("".join(res_str))
      

0 2.721181
euumu uuuuuueeeeeuuu
1 2.5610976
euem    eeeeeeeeeuuu
2 2.3250213
e mm     m          
3 2.579829
eummm meemeeeeeepppp
4 2.145481
eummm meemmeeeeeeuuu
5 1.8910974
eummm eeemm eeeeeuuu
6 1.7272816
eulmi eeem, eeeeeeup
7 1.5426176
eulmi eeem, ceeee-up
8 1.3142246
Mulmi teem, ceeer-up
9 1.0786749
Mulmi team, cheer-up
10 0.85138243
Mulmi team, cheer-up
11 0.6465982
Mulmi team, cheer-up
12 0.4574461
Mulmi team, cheer-up
13 0.3016736
Mulmi team, cheer-up
14 0.19306776
Mulmi team, cheer-up
15 0.1327713
Mulmi team, cheer-up
16 0.08265589
Mulmi team, cheer-up
17 0.05754142
Mulmi team, cheer-up
18 0.041446093
Mulmi team, cheer-up
19 0.030766327
Mulmi team, cheer-up
20 0.022163251
Mulmi team, cheer-up
21 0.015490632
Mulmi team, cheer-up
22 0.01119896
Mulmi team, cheer-up
23 0.008591381
Mulmi team, cheer-up
24 0.0068009174
Mulmi team, cheer-up
25 0.005350032
Mulmi team, cheer-up
26 0.0042072986
Mulmi team, cheer-up
27 0.0034087342
Mulmi team, cheer-up
28 0.0028605
Mulmi team, cheer-up
2

In [0]:
!pip install mlxtend
# apriori 알고리즘 탑재된 패키지

In [0]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder # encoder
from mlxtend.frequent_patterns import apriori # apriori algorithm

dataset = [['사과', '치즈', '생수'],
          ['생수','호두','치즈','고등어'],
          ['수박','사과','생수'],
          ['생수','호두','치즈','옥수수']]

In [0]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset) # 구매한거 : True, 안 구매한 것 : False

In [186]:
te_ary

array([[False,  True,  True, False, False,  True, False],
       [ True, False,  True, False, False,  True,  True],
       [False,  True,  True,  True, False, False, False],
       [False, False,  True, False,  True,  True,  True]])

In [187]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,고등어,사과,생수,수박,옥수수,치즈,호두
0,False,True,True,False,False,True,False
1,True,False,True,False,False,True,True
2,False,True,True,True,False,False,False
3,False,False,True,False,True,True,True


In [190]:
te.columns_mapping_ # index도 확인 가능

{'고등어': 0, '사과': 1, '생수': 2, '수박': 3, '옥수수': 4, '치즈': 5, '호두': 6}

In [196]:
freq_item = apriori(df, min_support=0.5, use_colnames=True) # 최소지지도 0.5
freq_item

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(사과, 생수)"
5,0.75,"(치즈, 생수)"
6,0.50,"(호두, 생수)"
7,0.50,"(호두, 치즈)"
8,0.50,"(치즈, 호두, 생수)"


In [222]:
from mlxtend.frequent_patterns import association_rules
association_rules(freq_item, metric='confidence', min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(사과),(생수),0.50,1.00,0.50,1.00,1.000000,0.000,inf
1,(치즈),(생수),0.75,1.00,0.75,1.00,1.000000,0.000,inf
2,(생수),(치즈),1.00,0.75,0.75,0.75,1.000000,0.000,1.0
3,(호두),(생수),0.50,1.00,0.50,1.00,1.000000,0.000,inf
4,(호두),(치즈),0.50,0.75,0.50,1.00,1.333333,0.125,inf
5,"(호두, 치즈)",(생수),0.50,1.00,0.50,1.00,1.000000,0.000,inf
6,"(호두, 생수)",(치즈),0.50,0.75,0.50,1.00,1.333333,0.125,inf
7,(호두),"(생수, 치즈)",0.50,0.75,0.50,1.00,1.333333,0.125,inf


# Mission : bc data

In [0]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/wisc_bc_data.csv")

In [224]:
data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.300100,0.147100,0.2419,0.07871,1.0950,0.9053,8.5890,153.400,0.006399,0.049040,0.053730,0.015870,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.71190,0.26540,0.4601,0.11890
1,842517,M,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.086900,0.070170,0.1812,0.05667,0.5435,0.7339,3.3980,74.080,0.005225,0.013080,0.018600,0.013400,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.24160,0.18600,0.2750,0.08902
2,84300903,M,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.197400,0.127900,0.2069,0.05999,0.7456,0.7869,4.5850,94.030,0.006150,0.040060,0.038320,0.020580,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.45040,0.24300,0.3613,0.08758
3,84348301,M,11.420,20.38,77.58,386.1,0.14250,0.28390,0.241400,0.105200,0.2597,0.09744,0.4956,1.1560,3.4450,27.230,0.009110,0.074580,0.056610,0.018670,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.68690,0.25750,0.6638,0.17300
4,84358402,M,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.198000,0.104300,0.1809,0.05883,0.7572,0.7813,5.4380,94.440,0.011490,0.024610,0.056880,0.018850,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.40000,0.16250,0.2364,0.07678
5,843786,M,12.450,15.70,82.57,477.1,0.12780,0.17000,0.157800,0.080890,0.2087,0.07613,0.3345,0.8902,2.2170,27.190,0.007510,0.033450,0.036720,0.011370,0.02165,0.005082,15.470,23.75,103.40,741.6,0.17910,0.52490,0.53550,0.17410,0.3985,0.12440
6,844359,M,18.250,19.98,119.60,1040.0,0.09463,0.10900,0.112700,0.074000,0.1794,0.05742,0.4467,0.7732,3.1800,53.910,0.004314,0.013820,0.022540,0.010390,0.01369,0.002179,22.880,27.66,153.20,1606.0,0.14420,0.25760,0.37840,0.19320,0.3063,0.08368
7,84458202,M,13.710,20.83,90.20,577.9,0.11890,0.16450,0.093660,0.059850,0.2196,0.07451,0.5835,1.3770,3.8560,50.960,0.008805,0.030290,0.024880,0.014480,0.01486,0.005412,17.060,28.14,110.60,897.0,0.16540,0.36820,0.26780,0.15560,0.3196,0.11510
8,844981,M,13.000,21.82,87.50,519.8,0.12730,0.19320,0.185900,0.093530,0.2350,0.07389,0.3063,1.0020,2.4060,24.320,0.005731,0.035020,0.035530,0.012260,0.02143,0.003749,15.490,30.73,106.20,739.3,0.17030,0.54010,0.53900,0.20600,0.4378,0.10720
9,84501001,M,12.460,24.04,83.97,475.9,0.11860,0.23960,0.227300,0.085430,0.2030,0.08243,0.2976,1.5990,2.0390,23.940,0.007149,0.072170,0.077430,0.014320,0.01789,0.010080,15.090,40.68,97.65,711.4,0.18530,1.05800,1.10500,0.22100,0.4366,0.20750


# Mission : grocery data

In [0]:
grolist = []

with open("/content/drive/My Drive/Colab Notebooks/groceries.csv") as f:
  for line in f.readlines():
    items = line.strip().split(",")
    grolist.append(items)

In [232]:
gro_data = grolist
gro_data

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese ', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product'],
 ['whole milk', 'butter', 'yogurt', 'rice', 'abrasive cleaner'],
 ['rolls/buns'],
 ['other vegetables',
  'UHT-milk',
  'rolls/buns',
  'bottled beer',
  'liquor (appetizer)'],
 ['pot plants'],
 ['whole milk', 'cereals'],
 ['tropical fruit',
  'other vegetables',
  'white bread',
  'bottled water',
  'chocolate'],
 ['citrus fruit',
  'tropical fruit',
  'whole milk',
  'butter',
  'curd',
  'yogurt',
  'flour',
  'bottled water',
  'dishes'],
 ['beef'],
 ['frankfurter', 'rolls/buns', 'soda'],
 ['chicken', 'tropical fruit'],
 ['butter', 'sugar', 'fruit/vegetable juice', 'newspapers'],
 ['fruit/vegetable juice'],
 ['packaged fruit/vegetables'],
 ['chocolate'],
 ['specialty bar'],
 ['other vegetables'],
 ['butter milk

In [0]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder # encoder
from mlxtend.frequent_patterns import apriori # apriori algorithm

In [234]:
te = TransactionEncoder()
te.fit(gro_data)

TransactionEncoder()

In [236]:
te_mat = te.transform(gro_data)
te_mat

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ...,  True, False, False],
       ...,
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [237]:
np.shape(te_mat)

(9835, 171)

In [242]:
te.columns_[:5]

['Instant food products',
 'UHT-milk',
 'abrasive cleaner',
 'artif. sweetener',
 'baby cosmetics']

In [244]:
te.columns_mapping_

{'Instant food products': 0,
 'UHT-milk': 1,
 'abrasive cleaner': 2,
 'artif. sweetener': 3,
 'baby cosmetics': 4,
 'baby food': 5,
 'bags': 6,
 'baking powder': 7,
 'bathroom cleaner': 8,
 'beef': 9,
 'berries': 10,
 'beverages': 11,
 'bottled beer': 12,
 'bottled water': 13,
 'brandy': 14,
 'brown bread': 15,
 'butter': 16,
 'butter milk': 17,
 'cake bar': 18,
 'candles': 19,
 'candy': 20,
 'canned beer': 21,
 'canned fish': 22,
 'canned fruit': 23,
 'canned vegetables': 24,
 'cat food': 25,
 'cereals': 26,
 'chewing gum': 27,
 'chicken': 28,
 'chocolate': 29,
 'chocolate marshmallow': 30,
 'citrus fruit': 31,
 'cleaner': 32,
 'cling film/bags': 33,
 'cocoa drinks': 34,
 'coffee': 35,
 'condensed milk': 36,
 'cooking chocolate': 37,
 'cookware': 38,
 'cream': 39,
 'cream cheese': 40,
 'cream cheese ': 41,
 'curd': 42,
 'curd cheese': 43,
 'decalcifier': 44,
 'dental care': 45,
 'dessert': 46,
 'detergent': 47,
 'dish cleaner': 48,
 'dishes': 49,
 'dog food': 50,
 'domestic eggs': 51,

In [241]:
df = pd.DataFrame(te_mat, columns=te.columns_)
df.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,...,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,sound storage medium,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False


In [271]:
freq_items = apriori(df, min_support=0.02, use_colnames=True).sort_values(by='support', ascending=False)
freq_items

,support,itemsets
57,0.255516,(whole milk)
39,0.193493,(other vegetables)
43,0.183935,(rolls/buns)
49,0.174377,(soda)
58,0.139502,(yogurt)
5,0.110524,(bottled water)
44,0.108998,(root vegetables)
53,0.104931,(tropical fruit)
47,0.098526,(shopping bags)
46,0.093950,(sausage)


In [274]:
from mlxtend.frequent_patterns import association_rules
association_rules(freq_items, metric='confidence', min_threshold=0).sort_values(by='lift', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
90,(root vegetables),"(whole milk, other vegetables)",0.108998,0.074835,0.023183,0.212687,2.842082,0.015026,1.175091
87,"(whole milk, other vegetables)",(root vegetables),0.074835,0.108998,0.023183,0.309783,2.842082,0.015026,1.290900
131,(tropical fruit),(pip fruit),0.104931,0.075648,0.020437,0.194767,2.574648,0.012499,1.147931
130,(pip fruit),(tropical fruit),0.075648,0.104931,0.020437,0.270161,2.574648,0.012499,1.226392
86,"(whole milk, root vegetables)",(other vegetables),0.048907,0.193493,0.023183,0.474012,2.449770,0.013719,1.533320
91,(other vegetables),"(whole milk, root vegetables)",0.193493,0.048907,0.023183,0.119811,2.449770,0.013719,1.080555
9,(other vegetables),(root vegetables),0.193493,0.108998,0.047382,0.244877,2.246605,0.026291,1.179941
8,(root vegetables),(other vegetables),0.108998,0.193493,0.047382,0.434701,2.246605,0.026291,1.426693
100,(yogurt),"(whole milk, other vegetables)",0.139502,0.074835,0.022267,0.159621,2.132979,0.011828,1.100890
97,"(whole milk, other vegetables)",(yogurt),0.074835,0.139502,0.022267,0.297554,2.132979,0.011828,1.225003
